In [1]:
!apt-get install -y coinor-cbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 21 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 http:/

In [30]:
from pyomo.environ import *

def bin_packing(weights, bin_capacity):
    n = len(weights)

    model = ConcreteModel()

    model.x = Var(range(n), range(n), within=Binary)
    model.y = Var(range(n), within=Binary)

    model.obj = Objective(expr=sum(model.y[j] for j in range(n)), sense=minimize)

    model.item_assignment = ConstraintList()
    for i in range(n):
        model.item_assignment.add(sum(model.x[i, j] for j in range(n)) == 1)

    model.bin_capacity = ConstraintList()
    for j in range(n):
        model.bin_capacity.add(sum(weights[i] * model.x[i, j] for i in range(n)) <= bin_capacity * model.y[j])

    solver = SolverFactory('cbc')
    solver.options['maxNodes'] = 10000
    solver.options['sec'] = 1000
    solver.solve(model, tee=True)

    bin_assignment = [-1] * n
    used_bins = 0
    for j in range(n):
        if model.y[j].value == 1:
            used_bins += 1
            for i in range(n):
                if model.x[i, j].value == 1:
                    bin_assignment[i] = j

    print(f"Number of bins used: {used_bins}")
    print(f"Bin assignment: {bin_assignment}")

weights = [10, 20, 15, 25, 30, 5, 10, 20]
bin_capacity = 50
bin_packing(weights, bin_capacity)

Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 1000 -printingOptions all -import /tmp/tmp6f8koadq.pyomo.lp -stat=1 -solve -solu /tmp/tmp6f8koadq.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 1000
Option for printingOptions changed from normal to all
Presolve 16 (0) rows, 72 (0) columns and 136 (0) elements
Statistics for presolved model
Original problem has 72 integers (72 of which binary)
==== 64 zero objective 2 different
64 variables have objective of 0
8 variables have objective of 1
==== absolute objective values 2 different
64 variables have objective of 0
8 variables have objective of 1
==== for integers 64 zero objective 2 different
64 variables have objective of 0
8 variables have objective of 1
==== for integers absolute objective values 2 different
64 variables have objective of 0
8 variables have objective of 1
===== end objective c

In [17]:
import os

def read_bpp(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    n = int(lines[0].strip())           # Number of items
    c = int(lines[1].strip())           # Bin capacity

    #item weights from the remaining lines
    weights = [int(line.strip()) for line in lines[2:]]

    if len(weights) != n:
        raise ValueError(f"Mismatch")
    return n, c, weights


def read_all():
    base_dir = '/content/drive/MyDrive/Scholl_1'
    file_prefix = 'N1C1W1_'
    file_count = 20

    bpp_data = {}

    for i in range(file_count):
        file_name = f"{file_prefix}{chr(65+i).zfill(1)}.txt"
        file_path = os.path.join(base_dir, file_name)

        if os.path.exists(file_path):
            print(f"Reading file: {file_path}")
            n, c, weights = read_bpp(file_path)
            bpp_data[file_name] = {'n': n, 'capacity': c, 'weights': weights}
        else:
            print(f"File not found: {file_path}")

    return bpp_data

In [18]:
data1 = read_all()

Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_A.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_B.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_C.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_D.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_E.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_F.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_G.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_H.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_I.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_J.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_K.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_L.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_M.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_N.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_O.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_P.txt
Reading file: /content/drive/MyDrive/Scholl_1/N1C1W1_Q.t

In [19]:
for file_name,file_data in data1.items():
  print(f"{file_name}")
  c=file_data['capacity']
  b=file_data['weights']
  result = bin_packing(b, c)
  if result:
    print(result)
    print()

N1C1W1_A.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmpoar29t0x.pyomo.lp -stat=1 -solve -solu /tmp/tmpoar29t0x.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== for integers 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== for integers absolute objective values 2 different
2500 variables have objective of 0
50 variables have o

Number of bins used: 26
Bin assignment: [6, 46, 49, 43, 26, 11, 10, 32, 14, 41, 12, 3, 25, 7, 28, 16, 24, 27, 29, 8, 4, 23, 19, 4, 23, 19, 2, 2, 24, 5, 1, 1, 8, 16, 5, 29, 7, 25, 5, 1, 3, 2, 1, 12, 8, 29, 32, 11, 19, 23]
N1C1W1_B.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmpcyd6clrz.pyomo.lp -stat=1 -solve -solu /tmp/tmpcyd6clrz.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== fo

Number of bins used: 32
Bin assignment: [21, 13, 35, 39, 10, 36, 25, 32, 8, 47, 46, 0, 5, 22, 34, 6, 1, 37, 42, 4, 3, 23, 9, 27, 33, 31, 15, 16, 7, 11, 11, 7, 12, 20, 12, 20, 27, 4, 1, 3, 23, 5, 37, 12, 20, 15, 23, 3, 37, 0]
N1C1W1_C.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmp4g_ko791.pyomo.lp -stat=1 -solve -solu /tmp/tmp4g_ko791.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 1
===

Number of bins used: 21
Bin assignment: [7, 13, 45, 18, 3, 9, 5, 0, 16, 27, 39, 8, 6, 46, 10, 14, 2, 4, 1, 1, 15, 2, 15, 4, 46, 20, 14, 6, 10, 20, 16, 20, 27, 5, 9, 15, 3, 20, 13, 14, 10, 18, 0, 4, 0, 2, 7, 0, 18, 1]
N1C1W1_D.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmp372v2rfx.pyomo.lp -stat=1 -solve -solu /tmp/tmp372v2rfx.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== for in

Number of bins used: 28
Bin assignment: [11, 37, 38, 8, 34, 32, 3, 25, 0, 9, 45, 20, 6, 16, 13, 36, 28, 24, 15, 46, 23, 7, 33, 1, 49, 35, 17, 47, 17, 35, 15, 28, 6, 47, 23, 49, 13, 20, 1, 49, 24, 24, 23, 9, 9, 1, 1, 13, 13, 38]
N1C1W1_E.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmp5noo0hsg.pyomo.lp -stat=1 -solve -solu /tmp/tmp5noo0hsg.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 1


...

Number of bins used: 27
Bin assignment: [9, 16, 20, 8, 19, 3, 0, 13, 4, 48, 5, 18, 6, 1, 10, 12, 28, 30, 42, 25, 40, 23, 15, 17, 2, 11, 2, 11, 12, 22, 1, 22, 10, 17, 6, 15, 22, 18, 23, 4, 13, 3, 0, 19, 17, 20, 15, 8, 16, 9]
N1C1W1_Q.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmp9_6hmpjl.pyomo.lp -stat=1 -solve -solu /tmp/tmp9_6hmpjl.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 1
====

Number of bins used: 28
Bin assignment: [28, 40, 27, 43, 47, 8, 18, 25, 38, 21, 13, 23, 26, 24, 2, 1, 12, 39, 29, 7, 3, 20, 11, 22, 17, 33, 14, 31, 14, 20, 3, 33, 22, 11, 39, 17, 7, 1, 12, 23, 26, 21, 13, 8, 47, 43, 18, 25, 27, 7]
N1C1W1_R.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmpxiolufbw.pyomo.lp -stat=1 -solve -solu /tmp/tmpxiolufbw.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of

Number of bins used: 27
Bin assignment: [30, 22, 47, 16, 23, 17, 11, 2, 41, 19, 33, 39, 5, 10, 48, 15, 0, 42, 1, 25, 7, 4, 21, 18, 7, 25, 4, 24, 21, 1, 24, 43, 3, 42, 5, 43, 3, 3, 43, 18, 18, 19, 24, 10, 11, 23, 43, 47, 17, 4]
N1C1W1_S.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmpeqyy1fgs.pyomo.lp -stat=1 -solve -solu /tmp/tmpeqyy1fgs.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 1
=

Number of bins used: 29
Bin assignment: [0, 46, 8, 30, 27, 42, 21, 29, 16, 47, 44, 14, 36, 23, 19, 38, 48, 40, 26, 34, 3, 1, 10, 25, 12, 45, 6, 45, 13, 2, 2, 6, 1, 10, 25, 13, 3, 12, 34, 48, 13, 26, 14, 23, 19, 2, 19, 23, 40, 42]
N1C1W1_T.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 100 -printingOptions all -import /tmp/tmp1awd7gsg.pyomo.lp -stat=1 -solve -solu /tmp/tmp1awd7gsg.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 100
Option for printingOptions changed from normal to all
Presolve 100 (0) rows, 2550 (0) columns and 5050 (0) elements
Statistics for presolved model
Original problem has 2550 integers (2550 of which binary)
==== 2500 zero objective 2 different
2500 variables have objective of 0
50 variables have objective of 1
==== absolute objective values 2 different
2500 variables have objective of 0
50 variables have objective of 

Number of bins used: 29
Bin assignment: [29, 37, 28, 20, 6, 16, 45, 9, 7, 1, 27, 3, 23, 10, 46, 40, 5, 25, 2, 18, 36, 0, 4, 34, 8, 22, 41, 33, 15, 15, 34, 33, 4, 8, 0, 18, 2, 41, 5, 36, 46, 40, 22, 22, 27, 3, 7, 23, 36, 23]


In [24]:
def first_fit_dec(n,c,l):
    bin_count = 0
    bins = [0]*n #worst case each item in separate bin
    bin_id=[0]*n
    l.sort(reverse=True)
    #print(l)
    for i in range(n):
        curr = 0
        while(curr < bin_count):
            if (bins[curr] >= l[i]):
                bins[curr] = bins[curr]-l[i]
                break
            curr+=1

        if (curr == bin_count):
            bins[bin_count] = c - l[i]
            bin_count+=1
        bin_id[i]=curr

    return bin_count, bin_id, l

In [25]:
for file_name,file_data in data1.items():
  print(f"{file_name}")
  c=file_data['capacity']
  b=file_data['weights']
  r1,r2,r3 = first_fit_dec(len(b),c,b)
  print(r1)
  print(r3)
  print("\n\n")

N1C1W1_A.txt
25
[99, 99, 96, 96, 92, 92, 91, 88, 87, 86, 85, 76, 74, 72, 69, 67, 67, 62, 61, 56, 52, 51, 49, 46, 44, 42, 40, 40, 33, 33, 30, 30, 29, 28, 28, 27, 25, 24, 23, 22, 21, 20, 17, 14, 13, 11, 10, 7, 7, 3]



N1C1W1_B.txt
31
[100, 99, 97, 97, 97, 93, 93, 92, 92, 88, 83, 83, 79, 76, 76, 75, 72, 71, 70, 69, 67, 66, 63, 62, 62, 61, 61, 51, 50, 44, 44, 43, 43, 40, 39, 37, 37, 30, 23, 20, 19, 18, 17, 15, 14, 13, 13, 12, 8, 8]



N1C1W1_C.txt
21
[92, 89, 87, 84, 82, 82, 81, 75, 73, 71, 67, 67, 63, 59, 57, 56, 52, 49, 48, 47, 46, 41, 39, 38, 36, 35, 34, 34, 30, 29, 26, 21, 20, 19, 18, 15, 15, 13, 11, 10, 10, 10, 9, 8, 8, 7, 6, 6, 6, 3]



N1C1W1_D.txt
28
[100, 99, 98, 97, 95, 94, 92, 92, 91, 82, 80, 77, 76, 75, 73, 73, 73, 71, 68, 65, 65, 63, 63, 63, 60, 59, 53, 45, 44, 40, 31, 25, 24, 24, 24, 23, 22, 21, 21, 15, 14, 14, 10, 10, 7, 7, 6, 3, 2, 2]



N1C1W1_E.txt
26
[91, 88, 88, 87, 87, 86, 86, 85, 85, 84, 83, 80, 79, 78, 77, 70, 70, 68, 67, 66, 59, 52, 49, 48, 47, 47, 44, 42, 38, 37, 

In [26]:
import os

def read_bpp(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    n = int(lines[0].strip())
    c = int(lines[1].strip())

    weights = [int(line.strip()) for line in lines[2:]]

    if len(weights) != n:
        raise ValueError(f"Mismatch")
    return n, c, weights


def read_all():
    base_dir = '/content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U'
    file_prefix = 'Falkenauer_u250_'
    file_count = 20

    bpp_data = {}

    for i in range(file_count):
        file_name = f"{file_prefix}{str(i).zfill(2)}.txt"
        file_path = os.path.join(base_dir, file_name)

        if os.path.exists(file_path):
            print(f"Reading file: {file_path}")
            n, c, weights = read_bpp(file_path)
            bpp_data[file_name] = {'n': n, 'capacity': c, 'weights': weights}
        else:
            print(f"File not found: {file_path}")

    return bpp_data

In [27]:
data = read_all()

Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_00.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_01.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_02.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_03.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_04.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_05.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_06.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_07.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_08.txt
Reading file: /content/drive/MyDrive/Falkenauer/Falkenauer/Falkenauer_U/Falkenauer_u250_09.txt
Reading file: /content/drive/MyDrive/Falkenauer/Fa

In [31]:
for file_name,file_data in data.items():
  print(f"{file_name}")
  c=file_data['capacity']
  b=file_data['weights']
  result = bin_packing(b, c)
  if result:
    print(result)
    print()

Falkenauer_u250_00.txt
Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -maxNodes 10000 -sec 1000 -printingOptions all -import /tmp/tmpk_9my81l.pyomo.lp -stat=1 -solve -solu /tmp/tmpk_9my81l.pyomo.soln (default strategy 1)
maxNodes was changed from 2147483647 to 10000
seconds was changed from 1e+100 to 1000
Option for printingOptions changed from normal to all
Presolve 500 (0) rows, 62750 (0) columns and 125250 (0) elements
Statistics for presolved model
Original problem has 62750 integers (62750 of which binary)
==== 62500 zero objective 2 different
62500 variables have objective of 0
250 variables have objective of 1
==== absolute objective values 2 different
62500 variables have objective of 0
250 variables have objective of 1
==== for integers 62500 zero objective 2 different
62500 variables have objective of 0
250 variables have objective of 1
==== for integers absolute objective values 2 different
62500 variables have objectiv

Number of bins used: 104
Bin assignment: [62, 7, 249, 132, 117, 86, 20, 26, 247, 41, 50, 92, 103, 1, 10, 121, 34, 17, 0, 90, 47, 82, 78, 45, 69, 153, 91, 4, 44, 89, 102, 112, 38, 88, 72, 49, 40, 16, 13, 12, 52, 24, 130, 11, 80, 81, 55, 28, 95, 59, 39, 105, 171, 37, 74, 77, 118, 73, 205, 56, 33, 60, 25, 57, 23, 163, 169, 128, 75, 137, 79, 110, 119, 64, 159, 27, 53, 54, 116, 14, 67, 98, 64, 85, 66, 58, 32, 205, 27, 77, 242, 159, 42, 37, 87, 154, 51, 74, 58, 95, 116, 63, 55, 48, 105, 54, 43, 59, 23, 137, 80, 104, 242, 12, 15, 169, 73, 6, 154, 6, 39, 89, 42, 40, 128, 87, 112, 66, 51, 85, 56, 60, 91, 8, 22, 24, 49, 25, 13, 121, 90, 84, 98, 47, 38, 163, 14, 33, 132, 20, 52, 48, 104, 43, 117, 92, 22, 57, 75, 249, 61, 63, 84, 15, 46, 22, 72, 103, 46, 44, 21, 247, 61, 153, 34, 10, 32, 2, 8, 46, 118, 53, 67, 2, 41, 171, 78, 28, 104, 11, 61, 102, 26, 88, 82, 110, 8, 2, 79, 69, 32, 86, 16, 43, 81, 50, 63, 21, 1, 110, 15, 62, 48, 79, 17, 130, 45, 130, 0, 67, 81, 53, 21, 21, 16, 171, 4, 28, 2, 84, 1

Number of bins used: 77
Bin assignment: [-1, -1, 0, -1, 1, 2, 3, -1, 4, -1, 248, 6, -1, 5, -1, 8, 7, -1, 10, -1, -1, -1, 13, -1, -1, -1, -1, 25, -1, -1, 15, 17, 18, 19, -1, 12, 23, 20, -1, 22, 16, -1, -1, 198, 24, 21, -1, 26, -1, -1, -1, -1, -1, 31, 34, -1, -1, -1, -1, 33, -1, 36, -1, -1, 28, -1, 35, 29, 32, 37, -1, 42, 44, 41, -1, 30, 43, 39, -1, -1, -1, 45, -1, -1, 40, -1, -1, 43, -1, 60, 50, 204, -1, -1, 47, -1, -1, 52, -1, -1, -1, 38, -1, 42, -1, 56, -1, 55, 49, -1, -1, -1, -1, 50, -1, -1, 41, 55, 57, 61, -1, 51, -1, -1, -1, 204, 70, -1, -1, -1, 46, 74, 57, 47, 59, 85, 60, 62, 51, -1, -1, -1, 49, 65, 72, 73, 54, 62, -1, -1, -1, 53, -1, -1, 74, -1, -1, 44, -1, -1, 68, 69, 72, -1, 66, 76, 61, -1, 73, -1, 59, -1, -1, -1, -1, -1, -1, 64, 71, 63, 64, 70, -1, 69, 71, -1, 69, -1, 71, -1, 85, 73, 65, 75, 75, 76, 63, 72, -1, 71, 75, 79, 76, 85, -1, 74, 64, -1, 67, -1, -1, -1, -1, -1, -1, -1, 65, 61, 70, 76, 79, -1, -1, 62, -1, -1, -1, -1, -1, -1, -1, 0, -1, 68, -1, 69, -1, -1, 75, 68, -1, -

Number of bins used: 77
Bin assignment: [-1, -1, 0, -1, 1, 2, 3, 4, -1, -1, -1, 6, -1, -1, 8, 9, -1, -1, 10, -1, -1, 12, 13, -1, -1, 11, -1, -1, -1, 87, 15, 22, 18, 14, 20, -1, -1, -1, 16, -1, 21, 23, 19, -1, -1, 27, -1, -1, -1, 33, 39, 28, -1, 24, -1, -1, -1, -1, 17, 25, -1, -1, -1, -1, -1, 32, 36, 31, -1, 34, 30, -1, 35, -1, -1, -1, 38, 45, -1, -1, -1, 37, -1, -1, 53, 41, 46, 49, -1, -1, 43, 48, -1, -1, 47, -1, 40, -1, -1, 42, 51, -1, 54, 46, -1, -1, 48, 50, 47, -1, -1, 56, -1, -1, 53, 55, 60, 52, 52, 58, 56, -1, 57, 59, -1, 63, -1, 44, -1, 67, 66, 83, 68, 50, 75, -1, 64, 65, 49, -1, 62, 74, 57, -1, 76, -1, 54, 60, 62, -1, 59, 67, -1, 79, -1, -1, 55, 63, 70, -1, -1, 70, -1, -1, -1, -1, -1, -1, -1, -1, 64, -1, 69, -1, -1, 66, -1, 68, 58, -1, -1, -1, 76, 61, -1, 74, 78, -1, 74, 75, -1, -1, -1, 79, -1, -1, -1, -1, 78, -1, -1, 83, 70, 80, 78, 79, -1, -1, -1, -1, -1, -1, -1, -1, 80, -1, 87, -1, 83, 67, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -

Number of bins used: 75
Bin assignment: [-1, -1, 0, 231, 1, 2, 3, -1, -1, -1, -1, 6, 110, 5, 8, 4, 10, 7, -1, -1, -1, -1, -1, -1, 15, -1, 12, 14, -1, 16, 9, -1, 17, 134, -1, 20, -1, -1, -1, -1, 28, 24, -1, 19, -1, 26, 18, -1, 22, 21, -1, -1, 30, 31, -1, -1, -1, -1, 27, -1, -1, -1, 32, 34, -1, 35, -1, -1, -1, 39, 41, 37, 36, -1, -1, -1, 38, 33, 43, 50, -1, 48, 32, -1, -1, 51, 39, 42, -1, 49, 37, 40, -1, 46, 44, 46, 52, 34, 45, -1, -1, 57, 44, 63, 43, 47, -1, 54, -1, -1, -1, 56, 53, -1, -1, 40, -1, -1, 51, 58, 61, -1, 47, 48, 49, -1, -1, -1, 54, 55, 50, 62, 59, 60, -1, 58, -1, -1, 71, 57, 55, 64, -1, 68, -1, 59, -1, -1, 62, 69, -1, -1, -1, -1, -1, 134, 67, -1, 60, 66, -1, -1, 64, -1, -1, -1, -1, -1, -1, 231, -1, -1, 64, -1, 52, 61, 66, 73, -1, -1, -1, -1, 71, 73, -1, 67, -1, 0, -1, -1, -1, -1, 76, 78, -1, 78, 60, 68, -1, 62, -1, -1, 63, -1, 72, -1, 65, -1, 71, 72, -1, -1, 67, 73, -1, 68, 76, -1, -1, -1, -1, -1, 72, -1, -1, -1, 72, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 76, -1, -1, -

...

Number of bins used: 83
Bin assignment: [-1, 105, -1, 1, 125, -1, 0, -1, 7, 5, -1, -1, -1, 10, -1, -1, -1, 12, -1, 11, -1, -1, -1, 133, 14, 6, 13, -1, -1, -1, 15, 18, 26, -1, -1, 19, 16, -1, 31, -1, 22, 24, 20, -1, 29, 28, -1, -1, 25, 17, -1, 27, -1, -1, 34, -1, 38, -1, 32, -1, -1, -1, 41, 33, -1, -1, -1, 35, 37, 39, 30, -1, 40, -1, -1, -1, 36, -1, -1, 42, 43, -1, -1, 45, 46, 50, -1, -1, 47, 59, 49, 52, -1, 51, 55, 56, -1, 60, 48, -1, 49, 54, -1, 44, 53, -1, -1, 52, -1, -1, 48, -1, 55, 39, -1, 40, 47, 50, 51, -1, 61, 60, 58, -1, -1, -1, 77, 68, -1, 56, -1, 62, 63, -1, 70, 41, 53, -1, 61, 62, -1, 65, 69, 54, 64, -1, 71, -1, 78, 66, -1, 67, -1, -1, 69, 75, -1, -1, 75, 58, -1, -1, -1, -1, 59, -1, 70, -1, -1, -1, -1, 73, 65, -1, 78, 4, -1, 82, 81, 71, -1, -1, -1, -1, 79, -1, -1, -1, -1, 88, -1, 83, 74, -1, 77, -1, -1, 79, 78, -1, -1, -1, -1, 30, -1, 77, -1, -1, -1, 79, 83, -1, -1, -1, -1, 73, 81, 68, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 83, -1, -1, -1, -1, -1, 0, -1, 82, -1, -1, -1,

Number of bins used: 112
Bin assignment: [54, 24, 78, 126, 25, 127, 50, 43, 247, 3, 1, 32, 168, 16, 71, 38, 8, 99, 30, 79, 90, 136, 107, 26, 61, 31, 229, 2, 81, 68, 9, 224, 13, 40, 28, 88, 39, 82, 65, 213, 106, 173, 4, 155, 77, 35, 52, 55, 12, 23, 5, 197, 20, 36, 41, 73, 0, 86, 6, 74, 103, 129, 27, 45, 169, 47, 62, 72, 34, 37, 49, 142, 233, 48, 69, 182, 46, 10, 138, 112, 176, 112, 42, 17, 51, 142, 102, 118, 69, 189, 149, 19, 182, 102, 49, 14, 47, 187, 73, 169, 0, 86, 118, 29, 34, 29, 80, 20, 12, 36, 67, 156, 9, 59, 61, 189, 28, 101, 37, 101, 68, 46, 138, 107, 21, 105, 55, 105, 5, 162, 59, 123, 162, 120, 7, 136, 51, 43, 66, 80, 18, 229, 60, 48, 94, 22, 155, 123, 35, 123, 67, 238, 24, 13, 6, 120, 238, 8, 19, 141, 108, 25, 60, 7, 22, 133, 42, 141, 168, 1, 90, 79, 141, 23, 77, 62, 22, 21, 108, 94, 7, 60, 27, 187, 17, 45, 41, 2, 3, 21, 133, 14, 129, 66, 17, 213, 66, 18, 72, 42, 72, 67, 10, 105, 108, 94, 176, 88, 4, 187, 31, 94, 18, 27, 66, 88, 149, 14, 176, 19, 31, 120, 129, 45, 238, 108, 4

Number of bins used: 79
Bin assignment: [-1, -1, 0, -1, 1, 2, 3, 4, -1, -1, 6, -1, -1, -1, 8, 9, -1, -1, -1, 10, 11, 12, 13, 14, -1, 84, -1, 17, -1, 20, -1, 22, -1, -1, -1, 16, 21, 15, -1, 23, -1, 27, -1, 25, 19, 24, -1, -1, -1, -1, -1, -1, -1, -1, 33, -1, -1, -1, 18, 32, 36, -1, 29, -1, 39, 40, -1, 35, -1, -1, 38, 34, 37, 31, 26, -1, 36, -1, 30, 43, -1, -1, 45, -1, 35, -1, 51, 44, 34, 49, 42, -1, 55, 39, 58, -1, 46, -1, -1, -1, 44, 47, 43, 42, -1, 54, 50, -1, 90, -1, 60, 41, -1, 53, -1, -1, -1, 40, 49, 61, 48, -1, 47, -1, 48, 51, 54, 59, 56, 64, 55, -1, -1, 50, 68, -1, 58, 62, 65, -1, 59, 63, 68, -1, 61, 66, 56, 52, 79, 75, -1, -1, 64, -1, -1, -1, -1, 57, -1, -1, -1, 74, -1, -1, -1, -1, -1, 52, 53, -1, -1, -1, -1, -1, 62, -1, 74, 71, -1, -1, -1, 66, -1, 62, 65, 76, 60, 63, -1, 67, 66, -1, -1, -1, 75, 83, -1, -1, 76, 74, 76, -1, -1, 79, 80, 80, 82, 82, 82, 83, 83, -1, -1, 82, -1, -1, -1, 70, 90, 90, -1, -1, -1, -1, -1, -1, -1, -1, 71, -1, 68, -1, -1, -1, -1, 65, 83, -1, -1, -1, -1, 84,

Number of bins used: 77
Bin assignment: [-1, -1, 0, -1, 1, 2, 3, 4, -1, -1, -1, 6, 10, 5, 8, 9, -1, -1, -1, 12, -1, -1, 13, -1, -1, 11, 80, 15, -1, 18, 17, -1, -1, -1, -1, -1, -1, -1, -1, -1, 16, 23, 20, 25, 21, -1, -1, -1, 33, -1, 26, -1, 30, 29, -1, 19, -1, -1, -1, 34, 37, -1, -1, -1, 31, 40, 24, -1, -1, -1, -1, 32, 36, 38, 41, 35, -1, 27, -1, 42, -1, 45, -1, 39, 129, -1, 44, -1, 48, 47, -1, -1, -1, 51, 46, 50, 43, -1, 43, 51, -1, 58, -1, 40, -1, 52, 42, 45, 44, -1, 56, -1, 62, 54, 49, -1, 47, 48, 57, 63, -1, -1, 49, -1, -1, 64, 59, -1, -1, 67, 58, -1, -1, -1, 74, -1, -1, -1, 66, -1, -1, -1, 64, -1, 54, -1, -1, -1, 226, 56, -1, -1, -1, 61, -1, 59, 68, 75, -1, -1, -1, -1, -1, -1, 62, -1, 53, -1, 76, 70, -1, -1, -1, -1, 79, 66, 52, 77, -1, 69, 77, -1, -1, 69, 59, 67, 70, 74, 60, 63, -1, 226, 73, 79, 83, -1, 77, -1, 75, 76, -1, 61, 64, -1, 68, 79, 73, 61, 77, -1, 69, -1, -1, -1, 83, -1, 75, -1, -1, 80, -1, -1, 79, 67, -1, -1, -1, -1, 75, -1, -1, 80, -1, -1, 23, -1, -1, -1, -1, 83, -1, 5

Number of bins used: 81
Bin assignment: [-1, -1, 0, 150, 1, 2, 3, -1, 134, -1, -1, 6, -1, 4, 8, -1, 5, 9, 10, -1, -1, 11, 13, -1, -1, -1, -1, 17, 16, 15, -1, 14, -1, -1, 26, -1, 23, -1, -1, 18, 177, -1, 28, -1, 24, 29, 19, -1, -1, 21, -1, -1, -1, 30, -1, -1, -1, 35, 33, 31, -1, 27, 34, -1, 37, 22, 44, 32, -1, -1, -1, -1, -1, -1, 42, -1, -1, -1, 42, 25, 48, -1, -1, 45, 40, -1, 37, 39, 40, 41, 43, -1, 47, -1, 61, -1, 51, -1, -1, -1, 53, 45, 32, 39, 59, 38, 47, 50, 56, -1, 66, -1, 43, -1, -1, -1, -1, -1, 79, 44, -1, 58, 50, -1, 41, 56, 64, 46, 53, -1, -1, 63, -1, 60, 54, 55, 67, 51, -1, 75, -1, 68, -1, -1, 48, -1, 65, -1, 59, 69, 57, -1, -1, -1, 134, 60, 62, -1, -1, 75, -1, 57, -1, 79, -1, 63, 66, 68, -1, 69, -1, -1, -1, -1, 65, -1, 83, -1, -1, 72, -1, 71, -1, -1, 77, -1, 67, 84, 73, 89, -1, -1, -1, 70, 62, 78, 55, 72, -1, 83, -1, -1, 78, 13, 67, 73, 65, -1, -1, 19, 84, 63, -1, -1, -1, -1, 89, -1, -1, 0, 73, 68, 72, 75, 69, 77, -1, 64, 59, 83, 84, 72, -1, -1, 71, -1, -1, 89, 89, 89, -1, -

In [29]:
for file_name,file_data in data.items():
  print(f"{file_name}")
  c=file_data['capacity']
  b=file_data['weights']
  r1,r2,r3 = first_fit_dec(len(b),c,b)
  print(r1)
  print(r3)
  print("\n\n")

Falkenauer_u250_00.txt
100
[100, 100, 100, 99, 99, 98, 98, 98, 98, 98, 98, 98, 98, 97, 97, 97, 96, 96, 95, 95, 95, 94, 94, 93, 93, 92, 92, 92, 91, 91, 90, 90, 90, 88, 88, 87, 86, 85, 85, 85, 84, 84, 84, 84, 84, 83, 83, 82, 82, 82, 81, 81, 81, 81, 80, 80, 80, 80, 80, 80, 79, 79, 79, 79, 78, 78, 78, 78, 78, 78, 76, 76, 75, 75, 74, 74, 74, 73, 73, 73, 73, 72, 72, 72, 71, 71, 70, 70, 70, 70, 70, 70, 69, 69, 69, 69, 68, 67, 67, 67, 67, 67, 66, 66, 66, 65, 65, 64, 64, 62, 62, 62, 61, 61, 60, 60, 60, 60, 60, 60, 59, 59, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 57, 55, 55, 55, 55, 55, 53, 53, 53, 53, 53, 53, 52, 52, 50, 50, 49, 49, 49, 49, 49, 48, 48, 47, 47, 47, 47, 46, 46, 46, 46, 45, 45, 45, 45, 45, 44, 44, 44, 43, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 42, 41, 41, 41, 41, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 37, 37, 36, 36, 36, 36, 36, 36, 35, 35, 33, 33, 33, 33, 32, 32, 32, 32, 30, 30, 30, 30, 30, 29, 29, 29, 28, 27, 27, 27, 27, 27, 26, 25, 25, 25, 24, 24, 24, 23, 23, 23, 23, 2